In [440]:
import os
import csv
from collections import defaultdict
import json

# To finetune

In [441]:
DATASET_VAR = 'orig'
DATASET_NAME = 'dataset'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TRAIN_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/train'
VALID_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/valid'
TEST_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/test'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'
FINAL_RESULTS_PATH = '/home/giacomoschiavo/finetuning-BirdNET/models/results'

In [442]:
print(f'python -m birdnet_analyzer.train --i {TRAIN_PATH} --o {RESULTS_PATH}/{DATASET_VAR}.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file {RESULTS_PATH}/{DATASET_VAR}.npz')

python -m birdnet_analyzer.train --i /home/giacomoschiavo/segments/dataset/train --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/orig/orig.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/orig/orig.npz


# To analize

In [443]:
MIN_CONF = '0.05'

In [444]:
print(f'python -m birdnet_analyzer.analyze --i {VALID_PATH} --o {RESULTS_PATH}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/valid --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/orig/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/orig/orig.tflite


In [445]:
print(f'python -m birdnet_analyzer.analyze --i {TEST_PATH} --o {RESULTS_PATH}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/test --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/orig/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/orig/orig.tflite


# Analysis

In [446]:
with open(f"./utils/{DATASET_NAME}/dataset_config_augm_final.json") as f:
    dataset_config = json.load(f)
class_names = list(dataset_config['mappings'].keys())

In [447]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [448]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(VALID_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [449]:
conf_scores = get_conf_scores()

In [450]:
from sklearn.metrics import f1_score
import numpy as np 

DEFAULT_THRESH = 0.15

def compute_best_thresholds(conf_scores, num_thresholds=200, min_thresh=0.01, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = DEFAULT_THRESH
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [451]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Phylloscopus collybita_Common Chiffchaff -> 0.067, F1-score: 0.524
📊 Sylvia atricapilla_Eurasian Blackcap -> 0.150, F1-score: 0.000
📊 Anthus trivialis_Tree Pipit -> 0.950, F1-score: 0.437
📊 Fringilla coelebs_Common Chaffinch -> 0.057, F1-score: 0.852
📊 Certhia familiaris_Eurasian Treecreeper -> 0.326, F1-score: 0.771
📊 Regulus regulus_Goldcrest -> 0.095, F1-score: 0.846
📊 Muscicapa striata_Spotted Flycatcher -> 0.150, F1-score: 0.000
📊 Coccothraustes coccothraustes_Hawfinch -> 0.907, F1-score: 0.947
📊 Erithacus rubecula_European Robin -> 0.223, F1-score: 0.439
📊 Periparus ater_Coal Tit -> 0.950, F1-score: 0.693
📊 Regulus ignicapilla_Common Firecrest -> 0.303, F1-score: 0.615
📊 Troglodytes troglodytes_Eurasian Wren -> 0.100, F1-score: 0.909
📊 Parus major_Great Tit -> 0.057, F1-score: 0.200
📊 Turdus merula_Eurasian Blackbird -> 0.123, F1-score: 0.286
📊 Lophophanes cristatus_Crested Tit -> 0.260, F1-score: 0.857
📊 Cuculus canorus_Common Cuckoo -> 0.150, F1-score: 0.000


In [452]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [453]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [454]:
true_segments_full.update(true_segments_test)

In [468]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = os.listdir(TEST_PATH)
# test_species_list = [species for species in test_species_list if len(species.split('_')) > 1]
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])
if DATASET_VAR == "orig":
    mlb.fit([[species for species in test_species_list if len(species.split("_")) > 1]])

len(mlb.classes_)

16

In [456]:
pred_segments_proba = {}
with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        # if len(file_path.split('/')[-2].split('_')) == 1:       # skip if None, Pecking, etc. 
        #     continue
        confidence = float(row['Confidence'])
        pred_segments_proba.setdefault(only_audio_name, {})
        best_thresholds.setdefault(pred_species_name, DEFAULT_THRESH)
        if confidence >= best_thresholds[pred_species_name]:
            pred_segments_proba[only_audio_name].setdefault(segm, {})
            if "None" in pred_segments_proba[only_audio_name][segm]:
                continue
            if pred_species_name == "None":
                pred_segments_proba[only_audio_name][segm] = {"None": confidence}
            else:
                pred_segments_proba[only_audio_name][segm].update({pred_species_name: confidence})
        else:
            pred_segments_proba[only_audio_name][segm] = {}

In [457]:
# extract recognized labels
pred_segments = {}
pred_proba = {}

for audio, segments in pred_segments_proba.items():
    pred_segments.setdefault(audio, {})
    pred_proba.setdefault(audio, {})
    for segm, labels in segments.items():
        pred_segments[audio].setdefault(segm, {})
        pred_segments[audio][segm] = list(labels.keys())
        pred_proba[audio].setdefault(segm, {})
        pred_proba[audio][segm] = list(labels.values())

In [458]:
true_segments = defaultdict(dict)
for audio in pred_segments.keys():
    for segm in pred_segments[audio].keys():
        if segm not in true_segments[audio_name]:
            true_segments[audio_name][segm] = []
        true_segments[audio][segm] = [species for species in true_segments_full[audio][segm] if species in test_species_list]

In [459]:
y_pred = []
y_true = []
y_pred_proba = []

for audio in pred_segments:
    for segment in sorted(pred_segments[audio].keys()):
        # Etichette vere e predette per questo segmento
        true_labels = true_segments[audio].get(segment, [])
        pred_labels = pred_segments[audio].get(segment, [])
        proba_values = pred_proba[audio].get(segment, [])

        # binarizzo con MultiLabelBinarizer
        y_true_vec = mlb.transform([true_labels])[0]  # 1D array
        y_pred_vec = mlb.transform([pred_labels])[0]  # 1D array

        # creo un vettore proba con zeri e ci metto i valori dove serve
        proba_vec = np.zeros(len(mlb.classes_))
        for label, score in zip(pred_labels, proba_values):
            if label in mlb.classes_:
                idx = list(mlb.classes_).index(label)
                proba_vec[idx] = score

        y_true.append(y_true_vec)
        y_pred.append(y_pred_vec)
        y_pred_proba.append(proba_vec)

# converto in array finali
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)


/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['None'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Aeroplane'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Vegetation'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Wind'] will be ignored
  warnings.warn(


In [463]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Anthus trivialis_Tree Pipit,1.000000,0.624242,0.768657,165.0
Certhia familiaris_Eurasian Treecreeper,0.959459,0.739583,0.835294,96.0
Coccothraustes coccothraustes_Hawfinch,0.857143,0.240000,0.375000,75.0
Erithacus rubecula_European Robin,0.928910,0.412632,0.571429,475.0
Fringilla coelebs_Common Chaffinch,0.909091,0.540788,0.678161,1091.0
Lophophanes cristatus_Crested Tit,1.000000,0.666667,0.800000,12.0
Loxia curvirostra_Common Crossbill,0.000000,0.000000,0.000000,15.0
Muscicapa striata_Spotted Flycatcher,0.750000,0.024390,0.047244,123.0
Parus major_Great Tit,0.111111,0.034483,0.052632,29.0
Periparus ater_Coal Tit,1.000000,0.119403,0.213333,134.0


In [464]:
with open(f'{RESULTS_PATH}/classification_report.json', 'w') as f:
    json.dump(report, f)

In [465]:
np.savez(f'{FINAL_RESULTS_PATH}/results_b{DATASET_VAR}.npz', y_true=y_true, y_pred=y_pred, y_pred_proba=y_pred_proba, class_names=class_names)